In [2]:
import pinocchio as pin
import numpy as np
import sys
import time
import os
from os.path import dirname, join, abspath
from pinocchio.utils import *
import crocoddyl
import matplotlib.pyplot as plt

from pinocchio.visualize import MeshcatVisualizer

In [3]:
# Load the URDF model.

#pinocchio_model_dir = dirname(dirname(str(abspath(__file__))))
pinocchio_model_dir = os.path.dirname(sys.executable)

model_path = join(pinocchio_model_dir, "example-robot-data/robots")
mesh_dir = pinocchio_model_dir


urdf_model_path='/Users/dkuzovlev/Documents/Robots/pinocchio-tutorial-master/dp_1_1.urdf'

model, collision_model, visual_model = pin.buildModelsFromUrdf(
    urdf_model_path, mesh_dir#, pin.JointModelFreeFlyer()
)
data = model.createData()

#urdf_model2_path='/Users/dkuzovlev/Documents/Robots/pinocchio-tutorial-master/dp_3_2.urdf'
#model2, collision_model2, visual_model2 = pin.buildModelsFromUrdf(urdf_model2_path, mesh_dir#, pin.JointModelFreeFlyer())
#data2 = model2.createData()


In [ ]:
#Meshcat initialisation (dependency: pip install meshcat)

try:
    viz = MeshcatVisualizer(model, collision_model, visual_model)

except ImportError as err:
    print(
            "Error while initializing the viewer. It seems you should install Python meshcat"
        )
    print(err)
    sys.exit(0)

viz.initViewer(open=True)

#viz.initViewer()
#hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()

In [ ]:
#start state
q0 = pin.neutral(model)
q1 = pin.randomConfiguration(model)
#v1 = np.random.randn(model.nv)
v1 = np.zeros(model.nv)

model.gravity.linear = np.array([0, 0, -9.78])

com = pin.centerOfMass(model, data, q1)

print("Center of Mass:", com)


In [6]:
#target state
q2 = pin.randomConfiguration(model)
v2 = np.random.randn(model.nv)
#q2 = q0
#v2 = np.zeros(model.nv)

In [7]:
# Load the robot in the viewer.
viz.loadViewerModel()
# Display a robot configuration.
viz.displayVisuals(True)
viz.display(q1)

viz.display(q2)
time.sleep(1)

viz.display(q1)
time.sleep(1)

In [ ]:
# Free motion simulation from q1 + visualisation

data = viz.data
             
model.gravity.linear = np.array([0, 0, -9.81])
dt = 0.01
 
 
def sim_loop():
    tau0 = np.zeros(model.nv) #Joint torques
    qs = [q1]
    vs = [v1] #Joint velocities
    
    nsteps = 10000
    for i in range(nsteps):
        q = qs[i]
        v = vs[i] 
        a1 = pin.aba(model, data, q, v, tau0) #joint acceleration (a).
        vnext = v + dt * a1
        qnext = pin.integrate(model, q, dt * vnext) # The integrate method takes the current joint configuration (q) and a small increment in the joint velocities (dq) and computes the new configuration (q'). This method allows you to move from one configuration to another based on the robot’s kinematics.
        qs.append(qnext)
        vs.append(vnext)
        viz.display(qnext)
        time.sleep(0.01)
        #viz.drawFrameVelocities(frame_id=frame_id)
    return qs, vs
qs, vs = sim_loop() 

In [ ]:
#Control from q1 to q2

# Create a Crocoddyl state and actuation model based on the Pinocchio model
state = crocoddyl.StateMultibody(model)
actuation = crocoddyl.ActuationModelFull(state)
running_cost_model = crocoddyl.CostModelSum(state)


goal_tracking_cost = crocoddyl.CostModelResidual(
    state,
    crocoddyl.ResidualModelState(state, pin.utils.np.concatenate((q2, v2)), actuation.nu)
)

control_cost = crocoddyl.CostModelResidual(
    state,
    crocoddyl.ResidualModelControl(state, actuation.nu)
)

running_cost_model.addCost("goal_tracking", goal_tracking_cost, weight=1)
#running_cost_model.addCost("control", control_cost, weight=0)

# Define the dynamics model using Pinocchio
dt = 0.01  # Time step
running_model = crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuation, running_cost_model)
running_integrated = crocoddyl.IntegratedActionModelEuler(running_model, dt)


# Define a terminal model (for reaching a goal)
terminal_model = crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuation, running_cost_model)
terminal_integrated = crocoddyl.IntegratedActionModelEuler(terminal_model, dt)

# Set the trajectory length and desired state/torque
#crocoddyl.ShootingProblem(initial_state, running_models, terminal_model)

T = 1000  # Number of time steps
problem = crocoddyl.ShootingProblem(pin.utils.np.concatenate((q1, v1)), [running_integrated] * T, terminal_integrated)

# Initialize the DDP solver
ddp = crocoddyl.SolverDDP(problem)

# Solve the optimization problem
ddp.solve()


In [ ]:
# print all controls (tau)
np.array(ddp.us)

In [ ]:
# plot tau (time)

plt.plot(np.array(ddp.us)[:, 0] , color ="red", label='tau1')
plt.plot(np.array(ddp.us)[:, 1] , color ="green", label='tau2')
plt.legend()
plt.show()

In [32]:
#model motion under control

data = viz.data             

model.gravity.linear = np.array([0, 0, -9.81])
 
def sim_loop():
    qs = [q1]
    vs = [v1] #Joint velocities
    

    for i in range(T):
        tau = ddp.us[i]
        q = qs[i]
        v = vs[i] 
        a1 = pin.aba(model, data, q, v, tau) #joint acceleration (a).
        vnext = v + dt * a1
        qnext = pin.integrate(model, q, dt * vnext) # The integrate method takes the current joint configuration (q) and a small increment in the joint velocities (dq) and computes the new configuration (q'). This method allows you to move from one configuration to another based on the robot’s kinematics.
        qs.append(qnext)
        vs.append(vnext)
        viz.display(qnext)
        time.sleep(0.01)
        #viz.drawFrameVelocities(frame_id=frame_id)
    return qs, vs

qs, vs = sim_loop() 
